In [15]:
import pandas as pd
import numpy as np

In [16]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [17]:
df = pd.read_csv("covid_toy.csv")

In [18]:
df.shape


(100, 6)

In [19]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [20]:
df.cough.value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [21]:
df.isnull().sum() # ab fever pr simple imputer chalain ge take null 
#data mean me  ya jo b ham usko instruction dain us me convert ho

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [22]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=["has_covid"]),df["has_covid"],test_size=0.2)

In [23]:
x_train

,age,gender,fever,cough,city
86,25,Male,104.0,Mild,Bangalore
77,8,Female,101.0,Mild,Kolkata
39,50,Female,103.0,Mild,Kolkata
59,6,Female,104.0,Mild,Kolkata
94,79,Male,NaN,Strong,Kolkata
...,...,...,...,...,...
19,42,Female,NaN,Strong,Bangalore
4,65,Female,101.0,Mild,Mumbai
11,65,Female,98.0,Mild,Mumbai
45,72,Male,99.0,Mild,Bangalore


In [24]:
df.city.value_counts


<bound method IndexOpsMixin.value_counts of 0       Kolkata
1         Delhi
2         Delhi
3       Kolkata
4        Mumbai
        ...    
95    Bangalore
96      Kolkata
97    Bangalore
98       Mumbai
99      Kolkata
Name: city, Length: 100, dtype: object>

## Aam zindagi

In [61]:
si= SimpleImputer()
x_train_fever=si.fit_transform(x_train[["fever"]])
x_test_fever=si.fit_transform(x_test[["fever"]])
x_train_fever.shape #ye null values me mean de ga by \default bc we use simple imputer

(80, 1)

In [32]:
oe=OrdinalEncoder(categories=[['Mild','Strong']])
x_train_cough=oe.fit_transform(x_train[['cough']])
x_test_cough=oe.fit_transform(x_test[['cough']])
x_train_cough.shape



(80, 1)

In [34]:
ohe=OneHotEncoder(drop="first",sparse=False)
x_train_gender_city=ohe.fit_transform(x_train[['gender','city']])
x_test_gender_city=ohe.fit_transform(x_test[['gender','city']])
x_train_gender_city.shape

D:\umama\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
D:\umama\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [36]:
x_train_gender_city #we dropped first columns of both. bcx it will not disturb our data
# when all will 0 it will automatically indicate the dropped one(which is one)

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 1.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.],
       [1., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 1.],


In [39]:
x_train_age=x_train.drop(columns=["gender","fever","cough","city"]).values
x_test_age=x_test.drop(columns=["gender","fever","cough","city"]).values
x_train_age.shape

(80, 1)

In [41]:
x_train_transformed=np.concatenate((x_train_age,x_train_fever,x_train_gender_city,x_train_cough),axis=1)
x_test_transformed=np.concatenate((x_test_age,x_test_fever,x_test_gender_city,x_test_cough),axis=1)
x_train_transformed.shape

(80, 7)

## Mentos zindagi

In [56]:
from sklearn.compose import ColumnTransformer

### transformer

In [63]:
transformer=ColumnTransformer(transformers=[
            ("tnf1",SimpleImputer(),["fever"]),#column k name ki jaga index bhi use hosakta he
            ("tnf2",OrdinalEncoder(categories=[['Mild','Strong']]),['cough'])
            ,("tnf3",OneHotEncoder(drop="first",sparse=False),['gender','city'])
                                                                                     ],
            remainder="passthrough" #why sparse is false?? take orignal me shange na ho
                          #drop colinearity khatam krne k liye ki he
            )


In [59]:
transformer.fit_transform(x_train).shape

D:\umama\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [60]:
transformer.fit_transform(x_test).shape

D:\umama\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(20, 7)